In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [52]:
import folium

## Choropleth map

In [5]:
df = pd.read_csv('data/world-happiness-report/2017.csv')
df.head()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [16]:
df.columns

Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual'],
      dtype='object')

In [6]:
maps = pd.read_csv('data/world-happiness-report/countries.csv')

In [7]:
maps.head()

,ISO 3166 Country Code,Country,Latitude,Longitude
0,AD,Andorra,42.50,1.50
1,AE,United Arab Emirates,24.00,54.00
2,AF,Afghanistan,33.00,65.00
3,AG,Antigua and Barbuda,17.05,-61.80
4,AI,Anguilla,18.25,-63.17


In [48]:
colorscale = [
    "#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9",
    "#9ecae1", "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be",
    "#2171b5", "#1361a9", "#08519c", "#0b4083", "#08306b"
]

data = dict(
    type='choropleth',
    colorscale=colorscale,
    autocolorscale=False,
    reversescale = True,
    locations=df['Country'],
    locationmode='country names',
    z=df['Economy..GDP.per.Capita.'],
    text=df['Country'],
    colorbar={'title': 'GDP per capita'})
layout = dict(
    title='Economy',
    geo=dict(showframe=False, projection={'type': 'Orthographic'}))

choromap = go.Figure(data=[data], layout=layout)
iplot(choromap)

From the above map we can see that GDP per capita is quite low in most parts of **Africa**, especially **central African countries**. **USA** and **European** countries have much better economy. **Saudi Arabia** stands out in middle east countries with GDP per capita very close to that of **USA**

## Symbols

In [24]:
df = df.merge(maps, how='left', on = ['Country'])
df.head()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,ISO 3166 Country Code,Latitude,Longitude
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027,NO,62.0,10.0
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707,DK,56.0,10.0
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715,IS,65.0,-18.0
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716,CH,47.0,8.0
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182,FI,64.0,26.0


In [49]:
df['text'] = df['Country'] + '<br>Happiness Score ' + (
    df['Happiness.Score']).astype(str)
colorscale = [
    "#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9",
    "#9ecae1", "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be",
    "#2171b5", "#1361a9", "#08519c", "#0b4083", "#08306b"
]

data = [
    dict(
        type='scattergeo',
        locationmode='country names',
        lon=df['Longitude'],
        lat=df['Latitude'],
        text=df['text'],
        mode='markers',
        marker=dict(
            size=df['Happiness.Score'] * 3,
            opacity=0.8,
            autocolorscale=False,
            reversescale = True,
            symbol='circle',
            line=dict(width=1, color='rgba(102, 102, 102)'),
            colorscale=colorscale,
            cmin=df['Economy..GDP.per.Capita.'].min(),
            color=df['Economy..GDP.per.Capita.'],
            cmax=df['Economy..GDP.per.Capita.'].max(),
            colorbar=dict(title="GDP per Capita")))
]

layout = dict(
    title='Relationship between happiness and economy',
    autosize=True,
    geo=dict(
        projection=dict(type='Mercator'),
        showland=True,
        landcolor="rgb(250, 250, 250)",
        countrywidth=0.7,
        subunitwidth=0.7),
)

choromap = go.Figure(data=data, layout=layout)

iplot(choromap)

Some things to observe:
* Countries with higher GDP per capita generally have high happiness score and ones with low GDP per capita have low happiness score
* Some anomalies: ***Botswana, Gabon*** have good economy but there happiness score is not high. Countries like ***Pakistan, Uzbekistan, Bhutan*** have high reported happiness score but have poor economy